In [3]:
import numpy as np

In [4]:
matrix1 = np.array([[1,2,3,4],
                    [2,3,4,5],
                    [3,4,5,6],
                    [4,5,6,7]])
matrix2 = np.array([[1,2,5,8],
                    [2,3,4,6],
                    [5,4,5,9],
                    [8,6,9,1]])
matrix3 = np.array([[2, -1, 0, -1],
                    [-1, 2, -1, 0],
                    [0, -1, 2, -1],
                    [-1, 0, -1, 4]])
np.linalg.det(matrix3)

8.000000000000002

In [5]:
#下三角偏移量-1
np.tril(matrix1, k=-1)
#上三角偏移量1
np.triu(matrix1, k=1)
#对角元素矩阵
np.diag(np.diag(matrix1))

array([[1, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 0, 5, 0],
       [0, 0, 0, 7]])

In [6]:
matrix1[1:] = matrix1[1:] + matrix1[1]
matrix1

array([[ 1,  2,  3,  4],
       [ 4,  6,  8, 10],
       [ 5,  7,  9, 11],
       [ 6,  8, 10, 12]])

In [7]:
matrix1[np.newaxis, 1:, 0] * matrix1[0, 1:, np.newaxis]

array([[ 8, 10, 12],
       [12, 15, 18],
       [16, 20, 24]])

In [8]:
#矩阵快速LDU分解
def quick_LDU(mat):
    '''
    该LDU分解需要传入方阵
    '''
    n = mat.shape[0]
    index = 0
    if np.linalg.det(mat) == 0:
        print("矩阵不可分解")
        return None
    mat = mat.copy().astype(np.float)
    def LDU(mat, index):
        '''
        通过n确定计算次数
        '''
        if index == n-1:
            L, D, U = np.tril(mat, k=-1), np.diag(np.diag(mat)), np.triu(mat, k=1)
            di = np.diag_indices(mat.shape[0])
            L[di], U[di] = 1, 1
            return L, D, U
        #开始该次的计算和赋值
#         print(mat[index+1:, index], mat[index, index+1:])
        #规格化计算
        mat[index, index+1:] = mat[index, index+1:] / mat[index, index]
        #对列消去
#         print(index,  mat, mat[index, index+1:] ,'and',  mat[index, index])
#         print((mat[np.newaxis, index+1:, index] * mat[index, index+1:, np.newaxis]))
        mat[index+1:, index+1:] = mat[index+1:, index+1:] - \
                                  (mat[np.newaxis, index+1:, index] * mat[index, index+1:, np.newaxis])
        #L赋值
        mat[index+1:, index] = mat[index, index+1:]  
        #函数递归调用
        index = index + 1
        return LDU(mat, index)
    
    return LDU(mat, index)
        
try:        
    L, D, U = quick_LDU(matrix3)
except Exception as e:
    print(e)

In [18]:
np.dot(L, np.dot(D, U)).astype(np.int8)

array([[ 2, -1,  0, -1],
       [-1,  2, -1,  0],
       [ 0, -1,  2, -1],
       [-1,  0, -1,  4]], dtype=int8)

In [17]:
U

array([[ 1.        , -0.5       ,  0.        , -0.5       ],
       [ 0.        ,  1.        , -0.66666667, -0.33333333],
       [ 0.        ,  0.        ,  1.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [16]:
#前代计算LZ = B
def forward_caculate(L, B):
    #将L对角线化为0
    L = L.copy()
    row = L.shape[0]
    L[np.diag_indices(row)] = 0
    Z = np.zeros((row,1))
    #判断B是不是稀疏向量
    if type(B) == int or np.sum(B == 0) == B.shape[0]-1:
        if type(B) == int:
            none_zero_index = B
        else:
            none_zero_index = np.flatnonzero(B)[0]
            B = B[none_zero_index,0]
        #非零元素以上行全为零
        Z[:none_zero_index-1,0] = 0
        for i in range(row - none_zero_index):
            if i == 0:
                Z[i+none_zero_index,0] =   B
            else:
                #print(L[np.newaxis, i+none_zero_index, none_zero_index:i+none_zero_index],Z[none_zero_index:i+none_zero_index,0, None])
                Z[i+none_zero_index,0] = -np.dot(L[np.newaxis, i+none_zero_index, none_zero_index:i+none_zero_index],Z[none_zero_index:i+none_zero_index,0, None])

    elif np.sum(B == 0) < B.shape[0]-1:
        for i in range(row):
            Z[i] = B[i] - np.dot(L[np.newaxis, i], Z)
            
    else:
        Z = np.zeros((row,1))
    
    return Z
B = np.array([0,1,0,0])[:,None]
Z = forward_caculate(L, B)
Z

array([[0.        ],
       [1.        ],
       [0.66666667],
       [1.        ]])

In [20]:
#规格化计算DY = Z
Y = Z/np.diag(D)[:, None]
Y

array([[0.        ],
       [0.66666667],
       [0.5       ],
       [0.5       ]])

In [25]:
#回代计算UX = Y
def backward_caculate(U, Y):
    #将L对角线化为0
    U = U.copy()
    row = U.shape[0]
    U[np.diag_indices(row)] = 0
    X = np.zeros((row, 1))
    for i in range(row-1, -1, -1):
        X[i] = Y[i] - np.dot(U[np.newaxis, i], X)
    return X
X = backward_caculate(U, Y)
X

array([[1. ],
       [1.5],
       [1. ],
       [0.5]])